In [7]:
# import the packages

library(lme4)
library(MuMIn)

# Analysis on the Checkpoint data

In [8]:
# Import the data
data <- read.csv("/mnt/upramdya_data/MD/F1_Tracks/Datasets/241114_F1_Checkpoints.csv")

# Show the data
head(data)

,X,index,fly_exit_time,distance,adjusted_time,fly,flypath,experiment,Nickname,Brain.region,⋯,X.1,F1_condition,success_direction,FeedingState,Light,Crossing,Balltype,EndChamber,pretraining,unlocked
,<int>,<int>,<dbl>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<lgl>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
1,0,0,6929.345,10,11.7931,240924_F1_3mm_ends_Videos_Checked_arena2_Right,/mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena2/Right,240924_F1_3mm_ends_Videos_Checked,PR,Control,⋯,NA,pretrained,,,,NA,,,y,n
2,1,1,6929.345,25,NA,240924_F1_3mm_ends_Videos_Checked_arena2_Right,/mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena2/Right,240924_F1_3mm_ends_Videos_Checked,PR,Control,⋯,NA,pretrained,,,,NA,,,y,n
3,2,2,6929.345,35,NA,240924_F1_3mm_ends_Videos_Checked_arena2_Right,/mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena2/Right,240924_F1_3mm_ends_Videos_Checked,PR,Control,⋯,NA,pretrained,,,,NA,,,y,n
4,3,3,6929.345,50,NA,240924_F1_3mm_ends_Videos_Checked_arena2_Right,/mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena2/Right,240924_F1_3mm_ends_Videos_Checked,PR,Control,⋯,NA,pretrained,,,,NA,,,y,n
5,4,4,6929.345,60,NA,240924_F1_3mm_ends_Videos_Checked_arena2_Right,/mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena2/Right,240924_F1_3mm_ends_Videos_Checked,PR,Control,⋯,NA,pretrained,,,,NA,,,y,n
6,5,5,6929.345,75,NA,240924_F1_3mm_ends_Videos_Checked_arena2_Right,/mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena2/Right,240924_F1_3mm_ends_Videos_Checked,PR,Control,⋯,NA,pretrained,,,,NA,,,y,n


In [9]:
# Get all columns
colnames(data)

[1] "X"                 "index"             "fly_exit_time"    
 [4] "distance"          "adjusted_time"     "fly"              
 [7] "flypath"           "experiment"        "Nickname"         
[10] "Brain.region"      "Date"              "Genotype"         
[13] "Period"            "Peak"              "Orientation"      
[16] "Pretraining"       "Unlocked"          "X.1"              
[19] "F1_condition"      "success_direction" "FeedingState"     
[22] "Light"             "Crossing"          "Balltype"         
[25] "EndChamber"        "pretraining"       "unlocked"

In [27]:
# List variables to be used in the model
vars <- c("adjusted_time", "pretraining", "unlocked", "distance", "fly", "Date")

# Remove the rows with missing values in the variables of interest
data_clean <- data[complete.cases(data[vars]), ]

# Check what was removed
print(dim(data))
print(dim(data_clean))

# Remove all columns that are not in the vars list
data_clean <- data_clean[vars]

# Check the structure of the cleaned data
str(data_clean)


[1] 1544   27
[1] 733  27
'data.frame':	733 obs. of  6 variables:
 $ adjusted_time: num  11.8 1210.4 1967.3 378.5 519 ...
 $ pretraining  : chr  "y" "y" "y" "y" ...
 $ unlocked     : chr  "n" "n" "n" "n" ...
 $ distance     : int  10 10 25 10 25 35 50 60 10 25 ...
 $ fly          : chr  "240924_F1_3mm_ends_Videos_Checked_arena2_Right" "240924_F1_3mm_ends_Videos_Checked_arena8_Right" "240924_F1_3mm_ends_Videos_Checked_arena8_Right" "240924_F1_3mm_ends_Videos_Checked_arena3_Right" ...
 $ Date         : int  240924 240924 240924 240924 240924 240924 240924 240924 240924 240924 ...


In [29]:
sapply(data_clean, function(x) sum(is.na(x)))

adjusted_time   pretraining      unlocked      distance           fly 
            0             0             0             0             0 
         Date 
            0

In [30]:
# Now run a linear mixed effects model to see if the adjusted time to reach each checkpoint is significantly different between the conditions, with important variables being: pretraining, unlocked, distance, success_direction and random effects being fly


model <- lmer(adjusted_time ~ pretraining * unlocked * distance + (1|fly) + (1|Date), 
              data = data_clean, 
              na.action = na.fail)

summary(model)

fixed-effect model matrix is rank deficient so dropping 2 columns / coefficients



Linear mixed model fit by REML ['lmerMod']
Formula: adjusted_time ~ pretraining * unlocked * distance + (1 | fly) +  
    (1 | Date)
   Data: data_clean

REML criterion at convergence: 12520

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.9675 -0.4848 -0.0290  0.4729  4.1826 

Random effects:
 Groups   Name        Variance Std.Dev.
 fly      (Intercept) 2236313  1495.4  
 Date     (Intercept)   93525   305.8  
 Residual             1031233  1015.5  
Number of obs: 733, groups:  fly, 147; Date, 14

Fixed effects:
                      Estimate Std. Error t value
(Intercept)           1327.894    277.419   4.787
pretrainingy          -395.013    387.482  -1.019
unlockedy             -241.974    366.413  -0.660
distance                42.318      2.818  15.019
pretrainingy:distance  -21.341      3.981  -5.361
unlockedy:distance       6.259      3.740   1.673

Correlation of Fixed Effects:
            (Intr) prtrnn unlckd distnc prtrn:
pretrainngy -0.667                    

In [31]:
model_selection <- dredge(model)

summary(model_selection)

Warning message in dredge(model):
“comparing models fitted by REML”
Fixed term is "(Intercept)"



fixed-effect model matrix is rank deficient so dropping 1 column / coefficient

fixed-effect model matrix is rank deficient so dropping 1 column / coefficient

fixed-effect model matrix is rank deficient so dropping 1 column / coefficient

fixed-effect model matrix is rank deficient so dropping 1 column / coefficient

fixed-effect model matrix is rank deficient so dropping 1 column / coefficient

fixed-effect model matrix is rank deficient so dropping 2 columns / coefficients



  (Intercept)        distance     pretraining unlocked  distance:pretraining
 Min.   : 986.7   Min.   :29.82   +   :14     +   :14   +   : 6             
 1st Qu.:1327.4   1st Qu.:29.92   NA's: 5     NA's: 5   NA's:13             
 Median :1737.1   Median :31.61                                             
 Mean   :1763.5   Mean   :35.57                                             
 3rd Qu.:1955.0   3rd Qu.:42.32                                             
 Max.   :2932.7   Max.   :42.32                                             
                  NA's   :5                                                 
 distance:unlocked pretraining:unlocked distance:pretraining:unlocked
 +   : 6           +   : 6              +   : 1                      
 NA's:13           NA's:13              NA's:18                      
                                                                     
                                                                     
                                  

In [32]:
top_models <- get.models(model_selection, subset = delta < 2)
model.avg(top_models)


fixed-effect model matrix is rank deficient so dropping 1 column / coefficient

fixed-effect model matrix is rank deficient so dropping 2 columns / coefficients




Call:
model.avg(object = top_models)

Component models: 
‘12345’   ‘123456’  ‘1234567’

Coefficients: 
       (Intercept) distance pretrainingy unlockedy distance:pretrainingy
full      1327.894 42.31775    -395.0133  -241.974             -21.34063
subset    1327.894 42.31775    -395.0133  -241.974             -21.34063
       distance:unlockedy
full             6.258761
subset           6.258761

Call:
model.avg(object = top_models)

Component models: 
‘12345’   ‘123456’  ‘1234567’

Coefficients: 
       (Intercept) distance pretrainingy unlockedy distance:pretrainingy
full      1327.894 42.31775    -395.0133  -241.974             -21.34063
subset    1327.894 42.31775    -395.0133  -241.974             -21.34063
       distance:unlockedy
full             6.258761
subset           6.258761

In [34]:
sw(model_selection)

distance pretraining distance:pretraining unlocked
Sum of weights:      1.00     1.00        1.00                 1.00    
N containing models:   14       14           6                   14    
                     distance:unlocked pretraining:unlocked
Sum of weights:      0.95              0.66                
N containing models:    6                 6                
                     distance:pretraining:unlocked
Sum of weights:      0.32                         
N containing models:    1

In [36]:
best_model <- lmer(adjusted_time ~ distance + pretraining + unlocked +
    distance:pretraining + distance:unlocked + pretraining:unlocked +
    (1 | fly) + (1 | Date), data=data_clean)

summary(best_model)


fixed-effect model matrix is rank deficient so dropping 1 column / coefficient



Linear mixed model fit by REML ['lmerMod']
Formula: 
adjusted_time ~ distance + pretraining + unlocked + distance:pretraining +  
    distance:unlocked + pretraining:unlocked + (1 | fly) + (1 |      Date)
   Data: data_clean

REML criterion at convergence: 12520

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.9675 -0.4848 -0.0290  0.4729  4.1826 

Random effects:
 Groups   Name        Variance Std.Dev.
 fly      (Intercept) 2236313  1495.4  
 Date     (Intercept)   93525   305.8  
 Residual             1031233  1015.5  
Number of obs: 733, groups:  fly, 147; Date, 14

Fixed effects:
                      Estimate Std. Error t value
(Intercept)           1327.894    277.419   4.787
distance                42.318      2.818  15.019
pretrainingy          -395.013    387.482  -1.019
unlockedy             -241.974    366.413  -0.660
distance:pretrainingy  -21.341      3.981  -5.361
distance:unlockedy       6.259      3.740   1.673

Correlation of Fixed Effects:
            (

# Summary of the model:

## Linear Mixed-Effects Model Analysis

### Model Overview

A linear mixed-effects model was fitted using the `lmer` function from the `lme4` package to analyze the adjusted time to reach each checkpoint. The model included fixed effects for distance, pretraining, unlocking, and their interactions, while accounting for random effects associated with individual flies and dates.

**Model Formula:**
$$
\text{adjusted\_time} \sim \text{distance} + \text{pretraining} + \text{unlocked} + \text{distance:pretraining} + \text{distance:unlocked} + \text{pretraining:unlocked} + (1 | \text{fly}) + (1 | \text{Date})
$$

### Model Fit

- **REML Criterion at Convergence:** 12520

### Residuals

The scaled residuals were analyzed:
- **Minimum:** -2.9675
- **1st Quartile:** -0.4848
- **Median:** -0.0290
- **3rd Quartile:** 0.4729
- **Maximum:** 4.1826

### Random Effects

The random effects structure indicated significant variability:

| Groups | Name        | Variance | Std. Dev. |
|--------|-------------|----------|-----------|
| fly    | (Intercept) | 2,236,313| 1495.4    |
| Date   | (Intercept) | 93,525   | 305.8     |
| Residual             | 1,031,233| 1015.5    |

- **Number of Observations:** 733
- **Groups:** 
  - Flies: 147
  - Dates: 14

### Fixed Effects

The fixed effects estimates are summarized below:

| Predictor                  | Estimate   | Std. Error | t value |
|----------------------------|------------|-------------|---------|
| (Intercept)                | 1327.894   | 277.419     | 4.787   |
| distance                   | 42.318     | 2.818       | 15.019  |
| pretraining (y)           | -395.013   | 387.482     | -1.019  |
| unlocked (y)               | -241.974   | 366.413     | -0.660  |
| distance:pretraining (y)   | -21.341    | 3.981       | -5.361  |
| distance:unlocked (y)      | 6.259      | 3.740       | 1.673   |

### Correlation of Fixed Effects

The correlation between fixed effects coefficients is as follows:

| Predictor                  | distance   | pretraining (y) | unlocked (y) |
|----------------------------|------------|------------------|--------------|
| distance                   | -0.389     |                  |              |
| pretraining (y)           | -0.667     | 0.278            |              |
| unlocked (y)               | 0.005      | -0.553           |              |
| distance:pretraining (y)   | 0.276      | -0.708           | -0.395       |
| distance:unlocked (y)      | 0.001      | 0.210            | -0.384       |

### Warnings

A warning was issued indicating that the fixed-effect model matrix is rank deficient, resulting in the dropping of one column/coefficient.

### Conclusion

The analysis indicates that:
- Distance significantly affects adjusted time.
- The interaction between distance and pretraining is significant, suggesting that pretraining modifies the effect of distance on adjusted time.
- The main effects of pretraining and unlocking alone are not statistically significant.
- There is substantial variability in adjusted time attributed to differences between individual flies and dates.

Further investigation may be warranted to address the rank deficiency and explore potential collinearity among predictors.